In [3]:
import numpy as np
import csv
import pandas as pd

from collections import Counter
from numpy import linalg as LA

In [4]:
with open("/content/drive/MyDrive/CIn/Mestrado/2021.1/Aprendizagem de Máquina/Projeto - Francisco/yeast.data") as input_file:
   lines = input_file.readlines()
   newLines = []
   for line in lines:
      newLine = line.strip().split()
      newLines.append(newLine)

with open("/content/drive/MyDrive/CIn/Mestrado/2021.1/Aprendizagem de Máquina/Projeto - Francisco/yeast.csv", 'w') as test_file:
   file_writer = csv.writer(test_file)
   file_writer.writerows(newLines)

In [5]:
# reading csv files
data =  pd.read_csv("/content/drive/MyDrive/CIn/Mestrado/2021.1/Aprendizagem de Máquina/Projeto - Francisco/yeast.csv",
                    names=["sequence_name", "mcg", "gvh", "alm", "mit", "erl", "pox", "vac", "nuc", "protein_local"])
print(data)

     sequence_name   mcg   gvh   alm   mit  erl  pox   vac   nuc protein_local
0       ADT1_YEAST  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22           MIT
1       ADT2_YEAST  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22           MIT
2       ADT3_YEAST  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22           MIT
3       AAR2_YEAST  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22           NUC
4       AATM_YEAST  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22           MIT
...            ...   ...   ...   ...   ...  ...  ...   ...   ...           ...
1479    YUR1_YEAST  0.81  0.62  0.43  0.17  0.5  0.0  0.53  0.22           ME2
1480    ZIP1_YEAST  0.47  0.43  0.61  0.40  0.5  0.0  0.48  0.47           NUC
1481    ZNRP_YEAST  0.67  0.57  0.36  0.19  0.5  0.0  0.56  0.22           ME2
1482    ZUO1_YEAST  0.43  0.40  0.60  0.16  0.5  0.0  0.53  0.39           NUC
1483    G6PD_YEAST  0.65  0.54  0.54  0.13  0.5  0.0  0.53  0.22           CYT

[1484 rows x 10 columns]


In [6]:
X = data.iloc[:, 1:9].values
y = data.iloc[:, 9].values

In [7]:
def p_estimate_parzen_window(x,h,X,y):

  classes = np.unique(y)

  concat = np.hstack((X,np.expand_dims(y, axis=1)))

  p_list = []

  for i in classes:

    input_values_class = concat[concat[:,-1]==i, 0:-1]

    sub = (x - input_values_class)/h

    cte = (1/(np.sqrt(2)*np.pi))

    K = np.prod(cte*np.exp(-sub.astype(float)**2/2),axis=1)

    p = sum(K)

    n_i = len(sub)

    d = len(sub[0])

    p_list.append([i,p/(n_i*(h**d))])
    
    p = 0

  return p_list

In [8]:
%%time

x = np.array([0.58, 0.61, 0.47, 0.13, 0.5, 0.0, 0.48, 0.22])

h = 0.1

print(p_estimate_parzen_window(x,h,X,y))

[['CYT', 69.33238643122478], ['ERL', 1.0379210962734828e-05], ['EXC', 28.406117524693766], ['ME1', 9.938011012213677], ['ME2', 37.504110365722426], ['ME3', 47.08696448872214], ['MIT', 43.180589385501904], ['NUC', 41.83642494029781], ['POX', 30.946992539259973], ['VAC', 60.555157564944054]]
CPU times: user 6.67 ms, sys: 1.94 ms, total: 8.61 ms
Wall time: 10.7 ms


In [9]:
def probability_priori(y):
  """return a list with each priori probability"""

  pr_pb = Counter(y)

  total = sum(pr_pb.values(), 0.0)

  for key in pr_pb:
    pr_pb[key] /= total  

  return pr_pb

In [18]:
class Parzen:
    def __init__(self, h=1):
        self.h = h

    def predict(self,x,X,y):

      pp = probability_priori(y)

      predict_parzen = []

      for x_i in x:

        parzen = p_estimate_parzen_window(x_i,self.h,X,y)

        parzen_probs = []

        for n,i in enumerate(parzen):
          num = pp[parzen[n][0]]*parzen[n][1]
          den = 0
          for k,j in enumerate(parzen):
            den+= pp[parzen[k][0]]*parzen[k][1]

          parzen_probs.append(num/den)

        predict_parzen.append(parzen[parzen_probs.index(max(parzen_probs))][0])

      return predict_parzen

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.neighbors import KNeighborsClassifier

skf = StratifiedKFold(n_splits=5)

kf = KFold(n_splits=5)

accuracy = []
precision = []
recall = []
f_measure = []

for train_index, test_index in skf.split(X, y):

  h_values = []

  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  for tr_index, tst_index in kf.split(X_train):

    parameter_h = []

    X_tr, X_val = X_train[tr_index], X_train[tst_index]
    y_tr, y_val = y_train[tr_index], y_train[tst_index]

    for h in np.linspace(0.01, 0.3, num=10):

      parzen_classifier = Parzen(h)
      parzen_predict = parzen_classifier.predict(X_val,X_tr,y_tr)

      parameter_h.append([accuracy_score(y_val, parzen_predict),h])

    h_values.append([max(parameter_h)[0],max(parameter_h)[1]])

  h = max(h_values)[1]

  print("Best h:",h)

  X_train = X[train_index]
  y_train = y[train_index]

  parzen_classifier = Parzen(h)
  parzen_predict = parzen_classifier.predict(X_test,X_train,y_train)

  accuracy.append(accuracy_score(y_test, parzen_predict))
  precision.append(precision_score(y_test, parzen_predict, average='macro',zero_division=0))
  recall.append(recall_score(y_test, parzen_predict, average='macro',zero_division=0))
  f_measure.append(f1_score(y_test, parzen_predict, average='macro',zero_division=0))

  print("accuracy: ",accuracy_score(y_test, parzen_predict))

print("\nMean accuracy: ",sum(accuracy)/len(accuracy), "Std: ",np.std(accuracy))
print("Mean precision: ",sum(precision)/len(precision), "Std: ",np.std(precision))
print("Mean recall: ",sum(recall)/len(recall), "Std: ",np.std(recall))
print("Mean f_measure: ",sum(f_measure)/len(f_measure), "Std: ",np.std(f_measure))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


Best h: 0.07444444444444444
accuracy:  0.5892255892255892
Best h: 0.07444444444444444
accuracy:  0.6094276094276094
Best h: 0.07444444444444444
accuracy:  0.5959595959595959
Best h: 0.07444444444444444
accuracy:  0.5084175084175084
Best h: 0.07444444444444444
accuracy:  0.5709459459459459

Mean accuracy:  0.5747952497952498 Std:  0.03542989209376387
Mean precision:  0.6139695939118838 Std:  0.04486153487879772
Mean recall:  0.5514757497622508 Std:  0.023195406520763937
Mean f_measure:  0.5586424899950423 Std:  0.02411368275613269
